### Data Preprocessing (TF-IDF) and Train/Test split
---

**Goal:** To pre-process data with TF-IDF and split into train and test sets.

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
import os
os.chdir("/content/gdrive/MyDrive/Colab/github/XMTC")

In [3]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', 500)
import matplotlib.pyplot as plt
%matplotlib inline
import csv

Read in tsv we prepared from previous notebook.

In [24]:
df = pd.read_csv('Task3preprocessed/netflix.tsv', sep='\t', index_col=0)
df.rename(columns={'description': 'plot_long'}, inplace=True)
df.head(10)

,title,rating,plot_long,Independent Movies,Faith & Spirituality,Documentaries,LGBTQ Movies,International TV Shows,TV Thrillers,TV Dramas,Stand-Up Comedy & Talk Shows,Thrillers,Anime Features,Science & Nature TV,TV Horror,Movies,Korean TV Shows,Teen TV Shows,Action & Adventure,Crime TV Shows,Anime Series,Cult Movies,Docuseries,Sci-Fi & Fantasy,TV Sci-Fi & Fantasy,Dramas,Sports Movies,TV Comedies,Horror Movies,Stand-Up Comedy,British TV Shows,Music & Musicals,TV Action & Adventure,Spanish-Language TV Shows,TV Mysteries,Reality TV,TV Shows,Comedies,Romantic TV Shows,Romantic Movies,Kids' TV,Classic Movies,International Movies,Classic & Cult TV,Children & Family Movies
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
s1,Dick Johnson Is Dead,PG-13,"As her father nears the end of his life, filmm...",0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
s2,Blood & Water,TV-MA,"After crossing paths at a party, a Cape Town t...",0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
s3,Ganglands,TV-MA,To protect his family from a powerful drug lor...,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
s4,Jailbirds New Orleans,TV-MA,"Feuds, flirtations and toilet talk go down amo...",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
s5,Kota Factory,TV-MA,In a city of coaching centers known to train I...,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
s6,Midnight Mass,TV-MA,The arrival of a charismatic young priest brin...,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
s7,My Little Pony: A New Generation,PG,Equestria's divided. But a bright-eyed hero be...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
s8,Sankofa,TV-MA,"On a photo shoot in Ghana, an American model s...",1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
s9,The Great British Baking Show,TV-14,A talented batch of amateur bakers face off in...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0


In order to differentiate my features from the potentially thousands of new features created by TF-IDF, I will add `g_` to the beginning of the genre columns, and `f_` to the beginning of the feature columns.

In [25]:
cols = list(df.columns.values)
genre_cols = cols[3:]
genre_cols = ['g_'+g for g in genre_cols]
print(len(genre_cols))
print(genre_cols)

42
['g_Independent Movies', 'g_Faith & Spirituality', 'g_Documentaries', 'g_LGBTQ Movies', 'g_International TV Shows', 'g_TV Thrillers', 'g_TV Dramas', 'g_Stand-Up Comedy & Talk Shows', 'g_Thrillers', 'g_Anime Features', 'g_Science & Nature TV', 'g_TV Horror', 'g_Movies', 'g_Korean TV Shows', 'g_Teen TV Shows', 'g_Action & Adventure', 'g_Crime TV Shows', 'g_Anime Series', 'g_Cult Movies', 'g_Docuseries', 'g_Sci-Fi & Fantasy', 'g_TV Sci-Fi & Fantasy', 'g_Dramas', 'g_Sports Movies', 'g_TV Comedies', 'g_Horror Movies', 'g_Stand-Up Comedy', 'g_British TV Shows', 'g_Music & Musicals', 'g_TV Action & Adventure', 'g_Spanish-Language TV Shows', 'g_TV Mysteries', 'g_Reality TV', 'g_TV Shows', 'g_Comedies', 'g_Romantic TV Shows', 'g_Romantic Movies', "g_Kids' TV", 'g_Classic Movies', 'g_International Movies', 'g_Classic & Cult TV', 'g_Children & Family Movies']


In [26]:
feature_cols = ['f_'+f for f in cols[:3]]
#feature_cols = ['f_'+cols[0], 'f_'+cols[2]]
feature_cols

['f_title', 'f_rating', 'f_plot_long']

In [27]:
#cols = feature_cols + genre_cols + ['plot_long']
cols = feature_cols + genre_cols

In [28]:
print(cols)

['f_title', 'f_rating', 'f_plot_long', 'g_Independent Movies', 'g_Faith & Spirituality', 'g_Documentaries', 'g_LGBTQ Movies', 'g_International TV Shows', 'g_TV Thrillers', 'g_TV Dramas', 'g_Stand-Up Comedy & Talk Shows', 'g_Thrillers', 'g_Anime Features', 'g_Science & Nature TV', 'g_TV Horror', 'g_Movies', 'g_Korean TV Shows', 'g_Teen TV Shows', 'g_Action & Adventure', 'g_Crime TV Shows', 'g_Anime Series', 'g_Cult Movies', 'g_Docuseries', 'g_Sci-Fi & Fantasy', 'g_TV Sci-Fi & Fantasy', 'g_Dramas', 'g_Sports Movies', 'g_TV Comedies', 'g_Horror Movies', 'g_Stand-Up Comedy', 'g_British TV Shows', 'g_Music & Musicals', 'g_TV Action & Adventure', 'g_Spanish-Language TV Shows', 'g_TV Mysteries', 'g_Reality TV', 'g_TV Shows', 'g_Comedies', 'g_Romantic TV Shows', 'g_Romantic Movies', "g_Kids' TV", 'g_Classic Movies', 'g_International Movies', 'g_Classic & Cult TV', 'g_Children & Family Movies']


Columns have been renamed. Let's apply this to our dataset

In [29]:
df.columns = cols

---

**Split Data**

Split the data into our X and y where X are the independent variables (everything except the genres), and y holds our target variables (just the genres).

In [30]:
# X (movie info and plot) and y (genres)
X = df.drop(genre_cols, axis=1)
y = df[genre_cols]

In [31]:
X

,f_title,f_rating,f_plot_long
id,,,
s1,Dick Johnson Is Dead,PG-13,"As her father nears the end of his life, filmm..."
s2,Blood & Water,TV-MA,"After crossing paths at a party, a Cape Town t..."
s3,Ganglands,TV-MA,To protect his family from a powerful drug lor...
s4,Jailbirds New Orleans,TV-MA,"Feuds, flirtations and toilet talk go down amo..."
s5,Kota Factory,TV-MA,In a city of coaching centers known to train I...
...,...,...,...
s8803,Zodiac,R,"A political cartoonist, a crime reporter and a..."
s8804,Zombie Dumb,TV-Y7,"While living alone in a spooky town, a young g..."
s8805,Zombieland,R,Looking to survive in a world taken over by zo...


---

**Pre-processing data (stem / lemmatize)**

Below are a series of functions to tokenize, stem, lemmatize and untokenize our text. All of these steps are need to 'clean' the text.

In [32]:
#Importing the packages we will be using
# Basic Packages
import numpy as np
import pandas as pd

# NLTK Packages
import nltk
# Use the code below to download the NLTK package, a straightforward GUI should pop up
# nltk.download()
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer

In [33]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [34]:
from nltk.tokenize import word_tokenize
def tokenizer(text):
    """
    Tokenizes the document
    """
    return word_tokenize(text)


from nltk.corpus import stopwords

#Load up our stop words
stop_words = stopwords.words('english')
#Adds stuff to our stop words list
stop_words.extend(['.',','])


## This function can improve, simplify. Look into Text Data Lecture
def remove_stopwords(list_of_tokens):
    """
    Removes stopwords
    """

    cleaned_tokens = []

    for token in list_of_tokens:
        if token in stop_words: continue
        cleaned_tokens.append(token)

    return cleaned_tokens


from nltk.stem import PorterStemmer

def stemmer(list_of_tokens):
    '''
    Takes in an input which is a list of tokens, and spits out a list of stemmed tokens.
    '''

    stemmed_tokens_list = []

    for i in list_of_tokens:

        token = PorterStemmer().stem(i)
        stemmed_tokens_list.append(token)

    return stemmed_tokens_list


from nltk.stem import WordNetLemmatizer

def lemmatizer(list_of_tokens):

    lemmatized_tokens_list = []

    for i in list_of_tokens:
        token = WordNetLemmatizer().lemmatize(i)
        lemmatized_tokens_list.append(token)

    return lemmatized_tokens_list


def the_untokenizer(token_list):
        '''
        Returns all the tokenized words in the list to one string.
        Used after the pre processing, such as removing stopwords, and lemmatizing.
        '''
        return " ".join(token_list)

def cleaning_our_texts(text):

    cleaned_texts = []
    num_texts = len(text)
    #for i in range(len(text)):
        # Check if the element is a string; if not, convert it to a string
        #that_string = str(text[i])

    for i in text.index:
        that_string = text[i]

        tokenized_list = tokenizer(that_string)
        removed_stopwords = remove_stopwords(tokenized_list)
        stemmed_words = stemmer(removed_stopwords)
        lemmatized_words = lemmatizer(stemmed_words)
        back_to_string = the_untokenizer(lemmatized_words)

        cleaned_texts.append(back_to_string)

    return cleaned_texts

In [35]:
%%time
X['plot_clean'] = cleaning_our_texts(X['f_plot_long'])

CPU times: user 7.61 s, sys: 38.5 ms, total: 7.65 s
Wall time: 7.69 s


In [37]:
#X = X.drop(['plot_long'],axis=1)
X = X.drop(['f_plot_long'],axis=1)
X

,f_title,f_rating,plot_clean
id,,,
s1,Dick Johnson Is Dead,PG-13,a father near end life filmmak kirsten johnson...
s2,Blood & Water,TV-MA,after cross path parti cape town teen set prov...
s3,Ganglands,TV-MA,to protect famili power drug lord skill thief ...
s4,Jailbirds New Orleans,TV-MA,feud flirtat toilet talk go among incarcer wom...
s5,Kota Factory,TV-MA,in citi coach center known train india ’ fines...
...,...,...,...
s8803,Zodiac,R,a polit cartoonist crime report pair cop inves...
s8804,Zombie Dumb,TV-Y7,while live alon spooki town young girl befrien...
s8805,Zombieland,R,look surviv world taken zombi dorki colleg stu...


Next to vectorize the clean text.

---

But first, SPLIT data now for testing later and to avoid leakage.

In [38]:
from sklearn.model_selection import train_test_split
# Split with 25% test size
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state = 123)

In [39]:
# NLP preprocessing
from sklearn.feature_extraction.text import TfidfVectorizer

# Instantiate the Vectorizer
# min_df at 20 - token should appear in at least 20 documents to be counted
tfidf = TfidfVectorizer(min_df=20) #, ngram_range=(1,3))

# Fit the vectorizer to the training data
tfidf.fit(X_train['plot_clean'])

# Transform both train and test sets
X_train_tfidf = tfidf.transform(X_train['plot_clean'])
X_test_tfidf = tfidf.transform(X_test['plot_clean'])

# pickle tfidf model for later use
import joblib
joblib.dump(tfidf, '../Category_Predict/models/netflix_tfidf_min20.pkl')

['models/netflix_tfidf_min20.pkl']

In [40]:
X_train_tfidf

<6605x1045 sparse matrix of type '<class 'numpy.float64'>'
	with 65024 stored elements in Compressed Sparse Row format>

In [41]:
X_test_tfidf

<2202x1045 sparse matrix of type '<class 'numpy.float64'>'
	with 21181 stored elements in Compressed Sparse Row format>

---

Random 30 words as a sample.

In [42]:
feature_names = tfidf.get_feature_names_out()

pd.DataFrame(feature_names).sample(30)

#pd.DataFrame(tfidf.get_feature_names()).sample(30)

,0
306,england
616,mysteri
266,disappear
1031,wonder
490,invad
559,lose
640,of
475,india
779,ruin
706,present


Change TD-IDF sparse matrix (for both train and test) back to dataframe format.

In [43]:
# Create new dataframe with the index of X_train, columns as the content of X_train_transformed token/feature names
feature_names = tfidf.get_feature_names_out()

# Create a DataFrame with the transformed data
X_train_transformed_df = pd.DataFrame(X_train_tfidf.toarray(), index=X_train.index, columns=feature_names)
print(X_train_transformed_df)

#X_train_transformed_df = pd.DataFrame(X_train_tfidf.toarray(), index=X_train.index, columns=tfidf.get_feature_names())
#X_train_transformed_df

       000   10   11  1970  1980   20   30  abandon  abil  abus  accept  \
id                                                                        
s4476  0.0  0.0  0.0   0.0   0.0  0.0  0.0      0.0   0.0   0.0     0.0   
s172   0.0  0.0  0.0   0.0   0.0  0.0  0.0      0.0   0.0   0.0     0.0   
s4795  0.0  0.0  0.0   0.0   0.0  0.0  0.0      0.0   0.0   0.0     0.0   
s5644  0.0  0.0  0.0   0.0   0.0  0.0  0.0      0.0   0.0   0.0     0.0   
s5646  0.0  0.0  0.0   0.0   0.0  0.0  0.0      0.0   0.0   0.0     0.0   
...    ...  ...  ...   ...   ...  ...  ...      ...   ...   ...     ...   
s7383  0.0  0.0  0.0   0.0   0.0  0.0  0.0      0.0   0.0   0.0     0.0   
s7764  0.0  0.0  0.0   0.0   0.0  0.0  0.0      0.0   0.0   0.0     0.0   
s5219  0.0  0.0  0.0   0.0   0.0  0.0  0.0      0.0   0.0   0.0     0.0   
s1347  0.0  0.0  0.0   0.0   0.0  0.0  0.0      0.0   0.0   0.0     0.0   
s3583  0.0  0.0  0.0   0.0   0.0  0.0  0.0      0.0   0.0   0.0     0.0   

       accid  accident  

In [44]:
# Create new dataframe with the index of X_test, columns as the content of X_test_transformed token/feature names
feature_names = tfidf.get_feature_names_out()

# Create a DataFrame with the transformed data
X_test_transformed_df = pd.DataFrame(X_test_tfidf.toarray(), index=X_test.index, columns=feature_names)
print(X_test_transformed_df)

#X_test_transformed_df = pd.DataFrame(X_test_tfidf.toarray(), index=X_test.index, columns=tfidf.get_feature_names())
#X_test_transformed_df

       000   10   11  1970  1980   20   30  abandon  abil  abus    accept  \
id                                                                          
s6427  0.0  0.0  0.0   0.0   0.0  0.0  0.0      0.0   0.0   0.0  0.000000   
s5910  0.0  0.0  0.0   0.0   0.0  0.0  0.0      0.0   0.0   0.0  0.392353   
s8001  0.0  0.0  0.0   0.0   0.0  0.0  0.0      0.0   0.0   0.0  0.000000   
s4544  0.0  0.0  0.0   0.0   0.0  0.0  0.0      0.0   0.0   0.0  0.000000   
s2953  0.0  0.0  0.0   0.0   0.0  0.0  0.0      0.0   0.0   0.0  0.000000   
...    ...  ...  ...   ...   ...  ...  ...      ...   ...   ...       ...   
s8193  0.0  0.0  0.0   0.0   0.0  0.0  0.0      0.0   0.0   0.0  0.000000   
s1406  0.0  0.0  0.0   0.0   0.0  0.0  0.0      0.0   0.0   0.0  0.000000   
s2063  0.0  0.0  0.0   0.0   0.0  0.0  0.0      0.0   0.0   0.0  0.000000   
s6988  0.0  0.0  0.0   0.0   0.0  0.0  0.0      0.0   0.0   0.0  0.000000   
s7898  0.0  0.0  0.0   0.0   0.0  0.0  0.0      0.0   0.0   0.0  0.000000   

Finally, add back on the numeric features for our complete `X_train`.

In [45]:
# Create final X_train dataframe which has been transformed, and only has numeric columns.
X_train_transformed_df_full = X_train.join(X_train_transformed_df, on=X_train_transformed_df.index)
X_train_transformed_df_full.shape

(6605, 1048)

In [46]:
X_test_transformed_df_full = X_test.join(X_test_transformed_df, on=X_test_transformed_df.index)
X_test_transformed_df_full.shape

(2202, 1048)

Last bit to do is put the `y` values back on to train and test sets and export to be imported in the modeling workbook.

Once joined there are some multi-level index issues to work out.

In [47]:
train_df_export = X_train_transformed_df_full.drop(['plot_clean'], axis=1).copy()
train_df_export = train_df_export.merge(y_train, on=train_df_export.index)
train_df_export = train_df_export.set_index(['key_0'])
train_df_export.reset_index(drop=True, inplace=True)

#del train_df_export.index.name
train_df_export

,f_title,f_rating,000,10,11,1970,1980,20,30,abandon,abil,abus,accept,accid,accident,account,accus,across,act,action,activist,actor,actress,adapt,addict,adopt,adult,adventur,affair,africa,african,after,ag,age,agent,ago,agre,aid,alien,all,alli,along,also,alter,alway,ambiti,america,american,amid,among,an,ancient,and,angel,anim,anoth,answer,antholog,anyth,apart,arm,armi,around,arrang,arriv,art,artist,ask,aspir,assassin,assist,at,athlet,attack,attempt,attend,attract,author,aveng,avoid,await,away,awkward,babi,bachelor,back,bad,band,bank,base,battl,bear,beauti,becom,befriend,begin,behind,believ,belov,best,betray,better,beyond,bheem,big,biggest,bill,biopic,birth,birthday,black,blind,bodi,bollywood,bond,book,born,bos,boy,boyfriend,brave,break,bring,british,brother,brutal,buddi,build,bulli,busi,businessman,but,call,camp,cancer,captur,car,care,career,cartel,case,cash,cast,cat,catch,caught,caus,celebr,center,centuri,challeng,champion,chanc,chang,chao,charact,charg,charm,chase,cheat,chef,child,childhood,chine,choic,choos,christma,chronicl,cia,citi,citizen,civil,claim,clash,class,classic,classmat,client,close,club,coach,collect,colleg,color,combat,come,comedi,comedian,comic,commit,common,commun,compani,compet,competit,complet,complex,complic,con,conflict,confront,connect,consequ,conspiraci,contest,control,controversi,convict,cook,cop,cope,corpor,corrupt,could,countri,coupl,cours,court,cover,crash,creat,creativ,creatur,crew,crime,crimin,crisi,cross,crush,cultur,curiou,dad,danc,dancer,danger,dare,dark,date,daughter,day,de,dead,deadli,deal,dealer,death,debt,decad,decid,deep,...,sinist,sister,six,skill,small,smart,soccer,social,societi,soldier,solv,someth,son,song,soon,soul,sound,south,space,spain,spark,special,spend,spi,spin,spirit,spiritu,sport,squad,stage,stand,star,start,state,station,stay,steal,step,still,stop,stori,strand,strang,stranger,street,strive,struggl,student,stumbl,success,suddenli,suffer,suicid,summer,super,superhero,supernatur,surpris,surviv,survivor,suspect,suspici,sweet,system,tackl,take,taken,tale,talent,talk,tangl,target,task,teach,teacher,team,tech,technolog,teen,teenag,tell,ten,tension,terrifi,terror,terrorist,test,the,thi,thief,thing,think,threat,threaten,three,thriller,throw,thrown,tim,time,to,togeth,top,torn,tough,tour,toward,town,trace,track,trade,tradit,traffick,tragedi,tragic,train,transform,trap,traumat,travel,treasur,tri,trial,triangl,trio,trip,troubl,true,truth,turmoil,turn,tv,twin,twist,two,unawar,uncl,uncov,undercov,underground,underworld,unexpect,unexpectedli,unfold,uniqu,unit,univers,unleash,unlik,unravel,up,upon,use,vacat,vanish,ve,version,veteran,victim,video,view,villag,villain,violenc,violent,visit,vow,wake,want,war,warrior,watch,water,way,wealthi,web,wed,weekend,well,west,when,while,white,whose,widow,wife,wild,will,win,wish,wit,with,without,woman,wonder,work,worker,world,would,write,writer,wrong,year,yet,york,young,younger,youth,g_Independent Movies,g_Faith & Spirituality,g_Documentaries,g_LGBTQ Movies,g_International TV Shows,g_TV Thrillers,g_TV Dramas,g_Stand-Up Comedy & Talk Shows,g_Thrillers,g_Anime Features,g_Science & Nature TV,g_TV Horror,g_Movies,g_Korean TV Shows,g_Teen TV Shows,g_Action & Adventure,g_Crime TV Shows,g_Anime Series,g_Cult Movies,g_Docuseries,g_Sci-Fi & Fantasy,g_TV Sci-Fi & Fantasy,g_Dramas,g_Sports Movies,g_TV Comedies,g_Horror Movies,g_Stand-Up Comedy,g_British TV Shows,g_Music & Musicals,g_TV Action & Adventure,g_Spanish-Language TV Shows,g_TV Mysteries,g_Reality TV,g_TV Shows,g_Comedies,g_Romantic TV Shows,g_Romantic Movies,g_Kids' TV,g_Classic Movies,g_International Movies,g_Classic & Cult TV,g_Children & Family Movies
0,Shirkers,TV-14,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.341585,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0

In [48]:
test_df_export = X_test_transformed_df_full.drop(['plot_clean'], axis=1).copy()
test_df_export = test_df_export.merge(y_test, on=test_df_export.index)
test_df_export = test_df_export.set_index(['key_0'])
test_df_export.reset_index(drop=True, inplace=True)
#del test_df_export.index.name
test_df_export

,f_title,f_rating,000,10,11,1970,1980,20,30,abandon,abil,abus,accept,accid,accident,account,accus,across,act,action,activist,actor,actress,adapt,addict,adopt,adult,adventur,affair,africa,african,after,ag,age,agent,ago,agre,aid,alien,all,alli,along,also,alter,alway,ambiti,america,american,amid,among,an,ancient,and,angel,anim,anoth,answer,antholog,anyth,apart,arm,armi,around,arrang,arriv,art,artist,ask,aspir,assassin,assist,at,athlet,attack,attempt,attend,attract,author,aveng,avoid,await,away,awkward,babi,bachelor,back,bad,band,bank,base,battl,bear,beauti,becom,befriend,begin,behind,believ,belov,best,betray,better,beyond,bheem,big,biggest,bill,biopic,birth,birthday,black,blind,bodi,bollywood,bond,book,born,bos,boy,boyfriend,brave,break,bring,british,brother,brutal,buddi,build,bulli,busi,businessman,but,call,camp,cancer,captur,car,care,career,cartel,case,cash,cast,cat,catch,caught,caus,celebr,center,centuri,challeng,champion,chanc,chang,chao,charact,charg,charm,chase,cheat,chef,child,childhood,chine,choic,choos,christma,chronicl,cia,citi,citizen,civil,claim,clash,class,classic,classmat,client,close,club,coach,collect,colleg,color,combat,come,comedi,comedian,comic,commit,common,commun,compani,compet,competit,complet,complex,complic,con,conflict,confront,connect,consequ,conspiraci,contest,control,controversi,convict,cook,cop,cope,corpor,corrupt,could,countri,coupl,cours,court,cover,crash,creat,creativ,creatur,crew,crime,crimin,crisi,cross,crush,cultur,curiou,dad,danc,dancer,danger,dare,dark,date,daughter,day,de,dead,deadli,deal,dealer,death,debt,decad,decid,deep,...,sinist,sister,six,skill,small,smart,soccer,social,societi,soldier,solv,someth,son,song,soon,soul,sound,south,space,spain,spark,special,spend,spi,spin,spirit,spiritu,sport,squad,stage,stand,star,start,state,station,stay,steal,step,still,stop,stori,strand,strang,stranger,street,strive,struggl,student,stumbl,success,suddenli,suffer,suicid,summer,super,superhero,supernatur,surpris,surviv,survivor,suspect,suspici,sweet,system,tackl,take,taken,tale,talent,talk,tangl,target,task,teach,teacher,team,tech,technolog,teen,teenag,tell,ten,tension,terrifi,terror,terrorist,test,the,thi,thief,thing,think,threat,threaten,three,thriller,throw,thrown,tim,time,to,togeth,top,torn,tough,tour,toward,town,trace,track,trade,tradit,traffick,tragedi,tragic,train,transform,trap,traumat,travel,treasur,tri,trial,triangl,trio,trip,troubl,true,truth,turmoil,turn,tv,twin,twist,two,unawar,uncl,uncov,undercov,underground,underworld,unexpect,unexpectedli,unfold,uniqu,unit,univers,unleash,unlik,unravel,up,upon,use,vacat,vanish,ve,version,veteran,victim,video,view,villag,villain,violenc,violent,visit,vow,wake,want,war,warrior,watch,water,way,wealthi,web,wed,weekend,well,west,when,while,white,whose,widow,wife,wild,will,win,wish,wit,with,without,woman,wonder,work,worker,world,would,write,writer,wrong,year,yet,york,young,younger,youth,g_Independent Movies,g_Faith & Spirituality,g_Documentaries,g_LGBTQ Movies,g_International TV Shows,g_TV Thrillers,g_TV Dramas,g_Stand-Up Comedy & Talk Shows,g_Thrillers,g_Anime Features,g_Science & Nature TV,g_TV Horror,g_Movies,g_Korean TV Shows,g_Teen TV Shows,g_Action & Adventure,g_Crime TV Shows,g_Anime Series,g_Cult Movies,g_Docuseries,g_Sci-Fi & Fantasy,g_TV Sci-Fi & Fantasy,g_Dramas,g_Sports Movies,g_TV Comedies,g_Horror Movies,g_Stand-Up Comedy,g_British TV Shows,g_Music & Musicals,g_TV Action & Adventure,g_Spanish-Language TV Shows,g_TV Mysteries,g_Reality TV,g_TV Shows,g_Comedies,g_Romantic TV Shows,g_Romantic Movies,g_Kids' TV,g_Classic Movies,g_International Movies,g_Classic & Cult TV,g_Children & Family Movies
0,Case Closed,TV-14,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.334681,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0

---
Finally, export our train and test sets, now with over 5000 features, to tsv files. These file are large - ~500MB for train and ~160MB for test.

In [49]:
%%time
train_df_export.to_csv('Task3preprocessed/netflix_train_dataframe.tsv', sep='\t')

CPU times: user 2.8 s, sys: 24.2 ms, total: 2.83 s
Wall time: 3.22 s


In [50]:
%%time
test_df_export.to_csv('Task3preprocessed/netflix_test_dataframe.tsv', sep='\t')

CPU times: user 886 ms, sys: 12.1 ms, total: 898 ms
Wall time: 943 ms
